
### Please ensure that this script is in the same directory as the text file (Mona_experimental_ms.txt)
Run the following code block in sequence, some code block might take up to half an hour (depending on computer speed)

In [14]:
import os
import glob
from tqdm import tqdm  #instant showing the progress of loops 
import csv
import pandas as pd
import numpy as np

In [26]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [27]:
os.getcwd()

'D:\\jupyter_workspace\\datacleaning\\GC-MS'

In [32]:
header = [
    'Name:', 
    'Formula:',
    'MW:',
    'NIST#:',
    'InChIKey:', 
    'MS2:',
    'Score:',
    'EstRI', #try to extract the 2 RI values in the text
    'PredRI',
]

In [33]:
txt_filename = 'GCMS-NIST20-example.txt' # name of the raw data text file
foldername = 'GCMS_output' # this is the folder that we are going to store our cleaned data
cleaned_folder_name = 'cleaned_data'
problematic_folder_name = 'problematic_data'
ignore = False # variable used to determine whether to exclude the element
file_exist = True # variable used to determine whether the file that we are going to append exist or not
repeating_key = False # variable used to determine whether there are multiple fields with same name in the element
item_dict = {} # dictionary to store all the fields and value for an element
string = '' # element in string form that are going to be written into error.txt when we can't decide its filename
filename = '' # name of the file to store the cleaned data
inchikey, mw= '','' # these variables are used to determine whether to include or exclude the element
ms2=[]
score=[]
msscore=[]

#trial on the ri index
PredRI = False #ri variable determine whether include AIri
EstRI = False # ri variable determine whether include Estri

if not (os.path.exists(foldername)):
    os.makedirs(foldername)

In [34]:
### This code block might take around 8 minutes to run
with open(txt_filename, mode='r', encoding='utf-8-sig') as readf:
    while True:
        row = readf.readline()

        if ('Name: 'in row or row ==''):
#             readf.readline()

            if not (ignore):
                # if there are multiple fields with same name in the current element,
                # we output them into repeating_key.csv for further debugging purpose
                if (repeating_key):
                    filename = os.path.join(os.getcwd(), 'repeating_key.csv')

                if (filename):

                    file_exist = os.path.isfile(filename)

                    with open(filename, 'a', encoding='utf-8-sig', newline='') as writef:
                        writer = csv.writer(writef)
                        if not (file_exist):
                            writer.writerow(header)
                        
                        ms2 = item_dict['MS2:']
                        score = item_dict['Score:']
                        while (len(ms2) > 0):
                            item_dict['MS2:'] = ms2.pop(0)
                            item_dict['Score:'] = score.pop(0)
                            writer.writerow(item_dict.get(col, '') for col in header)

                else:
                    # Will generate error.txt when:
                    # 2. There is trailing space or newline at the end of the file
                    with open('error.txt', 'a', encoding='utf-8-sig') as writef:
                        writef.write(string)
                        writef.write('\n')
                    
            # reinitialize everything after writing an item into a file
            ignore = False
            file_exist = True
            repeating_key = False
            item_dict = {}
            string = ''
            filename = ''
            ms2=[]
            score=[]
            msscore=[]
            EstRI = False
            PredRI = False
            
            if (row ==''):
                break # break out of the loop when we reach EOF
#             else:
#                 row = readf.readline()             
           
        # store the current element in string and dictionary data structure
        if('Name' in row):
            try:
                [key,val] = row.strip('\n').split(' ',1)
            except ValueError:
                print('ValueError: ')
                print(row)
                print(row.strip('\n').split(': '))
            try:
                if (key in item_dict.keys()):
                    item_dict[key] = [item_dict[key]]
                    item_dict[key].append(val)
                    repeating_key = True
                else:
                    item_dict[key] = val
            except IndexError:
                print('Index error: ', item_dict)

        if('Formula' in row):
            print(row)
            try:
                [key,val] = row.strip('\n').split(' ',1)
            except ValueError:
                print('ValueError: ')
                print(row)
                print(row.strip('\n').split(': '))
            try:
                if (key in item_dict.keys()):
                    item_dict[key] = [item_dict[key]]
                    item_dict[key].append(val)
                    repeating_key = True
                else:
                    item_dict[key] = val
            except IndexError:
                print('Index error: ', item_dict)

        if('MW: ' in row):
            print(row)
            try:
                [key,val] = row.strip('\n').split(' ')[0:2]
#                 print([key,val])
            except ValueError:
                print('ValueError: ')
                print(row)
                print(row.strip('\n').split(': '))
            try:
                if (key in item_dict.keys()):
                    item_dict[key] = [item_dict[key]]
                    item_dict[key].append(val)
                    repeating_key = True
                else:
                    item_dict[key] = val
            except IndexError:
                print('Index error: ', item_dict)

        if ('NIST#: ' in row):
            try:
                [key,val] = row.strip('\n').split(' ')[-6:-4]
#                 print([key,val])
            except ValueError:
                print('ValueError: ')
                print(row)
                print(row.strip('\n').split(': '))
            try:
                if (key in item_dict.keys()):
                    item_dict[key] = [item_dict[key]]
                    item_dict[key].append(val)
                    repeating_key = True
                else:
                    item_dict[key] = val
            except IndexError:
                print('Idex error: ', item_dict)
                
        if('InChIKey: ' in row):
            print(row)
            try:
                [key,val] = row.strip('\n').split(' ')[0:2]
#                 print([key,val])
            except ValueError:
                print('ValueError: ')
                print(row)
                print(row.strip('\n').split(': '))
            try:
                if (key in item_dict.keys()):
                    item_dict[key] = [item_dict[key]]
                    item_dict[key].append(val)
                    repeating_key = True
                else:
                    item_dict[key] = val
            except IndexError:
                print('Index error: ', item_dict)
        
        if('|' in row):  
            ms2=[]
            score=[]
            msscore=row.strip('\n\t')
            msscore=msscore.strip('')
            msscore2 = msscore.replace('\t',' ')
            msscore3 = msscore2.replace('|', '')
            msscore4 = msscore3.split(' ')
            msscore4 = [a for a in msscore4 if not a=='']
            for i in range(len(msscore4)):
                a=msscore4[i]
                if i%2==0:
                    ms2.append(a)
                else:
                    score.append(a) 
            if('MS2:' in item_dict.keys()):
                for item in ms2:
                    item_dict['MS2:'].append(item) 
            else:
                item_dict['MS2:']= ms2
            if('Score:' in item_dict.keys()):
                for item in score:
                    item_dict['Score:'].append(item)
            else:
                item_dict['Score:'] = score
#         print(item_dict)

        #extract the estimated RI and predicted RI
        if('Estimated non-polar retention index ' in row):
            EstRI = True
            continue
        
        if('Value:' in row):
            rowstart = row.strip('\n').split(' ')[0]
            if (rowstart =='Value:') & EstRI:
                print(row)
                #modified the key
                try:
                    riid = 'EstRI' 
                    rivalue = row.strip('\n').split(' ')[1]
                    [key,val] = [riid,rivalue]
#                     [key,val] = row.strip('\n').split(' ')[0:2]
                    print([key,val])
                except ValueError:
                    print('ValueError: ')
                    print(row)
                    print(row.strip('\n').split(': '))
                try:
                    if (key in item_dict.keys()):
                        item_dict[key] = [item_dict[key]]
                        item_dict[key].append(val)
                        repeating_key = True
                    else:
                        item_dict[key] = val
                except IndexError:
                    print('Index error: ', item_dict)
                EstRI = False
#             else:
#                 continue
    
        
        if('AI predicted non-polar retention index' in row):
            PredRI = True
            continue
        
        if('Value:' in row):
            rowstart = row.strip('\n').split(' ')[0]
            if (rowstart =='Value:') & PredRI:
                print(row)
                #modified the key
                try:
                    riid = 'PredRI' 
                    rivalue = row.strip('\n').split(' ')[1]
                    [key,val] = [riid,rivalue]
#                     [key,val] = row.strip('\n').split(' ')[0:2]
                    print([key,val])
                except ValueError:
                    print('ValueError: ')
                    print(row)
                    print(row.strip('\n').split(': '))
                try:
                    if (key in item_dict.keys()):
                        item_dict[key] = [item_dict[key]]
                        item_dict[key].append(val)
                        repeating_key = True
                    else:
                        item_dict[key] = val
                except IndexError:
                    print('Index error: ', item_dict)   
                PredRI = False
#             else:
#                 continue
            
        # exclude these fields
#         if ('Synon: ' in row or 'Comments: ' in row or 'Num Peaks: ' in row):
        if ('Contributor:' in row or 'Other DBs:'in row or'Synonyms: ' in row):
            continue
        if ('Values and Intensities: ' in row or'Confidence interval:'in row or 'Column Type:'in row):
            continue

       #determine the filename:
        if (not filename):
            if ('MW: ' in row):
                try:
                    filename = os.path.join(os.getcwd(), foldername, str(int(float(row.strip('\n').split(' ')[1]))) + '.csv')
                    mw = int(float(row.strip('\n').split(" ")[1]))
                except ValueError:
                    print('ValueError: ', row)
                    break

Formula: C6H16O4P2S

MW: 246 Exact Mass: 246.024453 NIST#: 383781 ID#: 220826 DB: mainlib

InChIKey: DNSXYZRDXBWQEH-UHFFFAOYSA-N   Non-stereo

Value: 1462 iu

['PredRI', '1462']
Formula: C10H6ClNO2

MW: 207 Exact Mass: 207.008706 CAS#: 2797-51-5 NIST#: 135389 ID#: 200731 DB: mainlib

InChIKey: OBLNWSCLAYSJJR-UHFFFAOYSA-N   Non-stereo

Value: 1891 iu

['EstRI', '1891']
Value: 2011 iu

['PredRI', '2011']
Formula: C10H6ClNO2

MW: 207 Exact Mass: 207.008706 CAS#: 2797-51-5 NIST#: 135389 ID#: 200731 DB: mainlib

InChIKey: OBLNWSCLAYSJJR-UHFFFAOYSA-N   Non-stereo

Value: 1891 iu

['EstRI', '1891']
Value: 2011 iu

['PredRI', '2011']
Formula: C10H6ClNO2

MW: 207 Exact Mass: 207.008706 CAS#: 2797-51-5 NIST#: 135389 ID#: 200731 DB: mainlib

InChIKey: OBLNWSCLAYSJJR-UHFFFAOYSA-N   Non-stereo

Value: 1891 iu

['EstRI', '1891']
Value: 2011 iu

['PredRI', '2011']
Formula: C11H14O3

MW: 194 Exact Mass: 194.094295 CAS#: 122-48-5 NIST#: 8502 ID#: 154129 DB: mainlib

InChIKey: OJYLAHXKWMRDGS-UHFFFAOYSA-

# extracting part finised. to the data cleanup and minning part

In [86]:
#define clean_data function to filter ms peaks with low abundance
#ms peak abundance minimum value 30

def clean_data(df, filename):
    # filter out rows with Score value < 30
    #NIST how to set the score value <30
    df = df[df['Score:'] >= 30]
    print('cleandata:', df.shape)#try to trouble shoot
    
    # output_df is the processed (cleaned) DataFrame that will be returned
    output_df = pd.DataFrame(columns=['Name:', 'Formula:', 'MW:', 'NIST#:', 'InChIKey:', 'MS2:', 'Score:','EstRI','PredRI'])    
    
    #sort inchikey to get the same compounds, sort ms2 to get ms2 for the same compounds
    df = df.sort_values(by=['InChIKey:', 'MS2:'], ascending=True)
    
    df_columns = ['Name:', 'Formula:', 'MW:', 'NIST#:']  
    inchikeys = df['InChIKey:'].unique().tolist()
    
    for inchikey in inchikeys:
        working_df = df[df['InChIKey:'] == inchikey]
        
#         if (working_df.shape[0] < 2):
#             # check for similar PrecursorMZ and MS2
#             output_df, problematic_df = df_check_precursorMZ_MS2(working_df, output_df, problematic_df)
#             continue
        
        output_MS = working_df.iloc[0]['MS2:'] # to store the chosen MS(MS with the highest score is chosen)
        output_MS_score = working_df.iloc[0]['Score:'] # to store the chosen MS score
        output_score = working_df.iloc[0]['Score:'] # to store the accumulated score
        retain_prev = False
        
        for i in range(1, working_df.shape[0]):
            # update prev_row and curr_row
            if not (retain_prev):
                prev_row = working_df.iloc[i-1].copy()
            curr_row = working_df.iloc[i]
            curr_row_MS = curr_row['MS2:']
            
            # check if both MS2 should be group together by using the formula
            # if they should be group together, we do the following:
            # 1. add up the score
            # 2. update output_MS2 to the one with the higher score
            # 3. check for all other columns other than (InChIKey, MS2, Score),
            if (abs(output_MS - curr_row_MS) <=1):
                retain_prev = True
                output_score += curr_row['Score:']
                if (curr_row['Score:'] > output_MS_score):
                    output_MS_score = curr_row['Score:']
                    output_MS = curr_row_MS
                for column in df_columns:
                    if (type(prev_row[column]) == list):
                        if (curr_row[column] not in prev_row[column]):
                            prev_row[column].append(curr_row[column])
                    elif (prev_row[column] != curr_row[column]):
                        prev_row[column] = [prev_row[column]]
                        prev_row[column].append(curr_row[column])
                        
            # if they shouldn't be group together, we update the MS2 and Score
            # to the chosen MS2 (output_MS2) and accumulated score (output_score)
            # and append the series to output_df
            else:
                retain_prev = False
                prev_row['MS2:'] = output_MS
                prev_row['Score:'] = output_score
                
                # check for similar PrecursorMZ and MS2
                output_df = output_df.append(prev_row, ignore_index=True)
#                 output_df = s_check_precursorMZ_MS(prev_row, output_df)
                output_MS2 = curr_row['MS2:']
                output_MS2_score = curr_row['Score:']
                output_score = curr_row['Score:']

        # handling the last row
        if (retain_prev):
            prev_row['MS2:'] = output_MS
            prev_row['Score:'] = output_score
            
            output_df = output_df.append(prev_row, ignore_index=True)
#             # check for similar PrecursorMZ and MS2
#             output_df = s_check_precursorMZ_MS(prev_row, output_df)
        else:
            output_df = output_df.append(curr_row, ignore_index=True)
#             # check for similar PrecursorMZ and MS2
#             output_df = s_check_precursorMZ_MS(curr_row, output_df)

    output_df.sort_values(by=['InChIKey:', 'Score:'], ascending=False, inplace=True)
#     return output_df, problematic_df
    print('outputdim:', output_df.shape)
    return output_df

In [89]:
#cleaning sorted data and prepare for the algorithm calculation
read_folder = os.path.join(os.getcwd(), foldername)  #folder that we will read data from
cleaned_folder = os.path.join(read_folder, 'cleaned_data') #folder to store cleaned data

if not (os.path.exists(cleaned_folder)):
    os.makedirs(cleaned_folder)

# get list of csv file names
cwd = os.getcwd()
os.chdir(read_folder)
listdir = glob.glob('*.csv')
os.chdir(cwd)

#for filename in listdir
for filename in tqdm(listdir):
    output_filename=os.path.join(cleaned_folder, filename.strip('.csv')+'_cleaned.csv')
    readf = os.path.join(read_folder, filename)
    
    try:
        df = pd.read_csv(readf)
    except OSError:
        print(readf)
    
#     df = df[['Name:', 'Formula:', 'MW:', 'NIST#:', 'InChIKey:', 'MS2:', 'Score:', 'EstRI' , 'PredRI']]
    df= clean_data(df,filename)
    
    if (df.shape[0]>0):
        df.to_csv(output_filename, header=True, mode='a', index = False)

100%|██████████| 4/4 [00:00<00:00, 25.26it/s]

cleandata: (20, 9)
outputdim: (1, 9)
cleandata: (138, 9)
outputdim: (1, 9)
cleandata: (55, 9)
outputdim: (1, 9)
cleandata: (46, 9)
outputdim: (1, 9)
